# ADM-HW3: GROUP #14

#### Xavier Del Giudice, Alessio Iacono, Geraldine Maurer


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
| Alessio Iacono |   ...   |                   ...                   |
| Geraldine Maurer |   ...   |                   ...                   |

## Import Packages 

In [1]:
import heapq
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import functions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from unidecode import unidecode
import string
import unicodedata
import pandas as pd
from collections import defaultdict
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
%pip install unidecode geopy plotly dash aiofiles aiohttp nltk ipywidgets requests bs4 pandas

In [1]:
import heapq
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import functions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from unidecode import unidecode
import string
import unicodedata
import pandas as pd
from collections import defaultdict
import pickle
import ipywidgets
import ipywidgets as ipw
from itertools import chain
import functions
from IPython.display import display

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Data Collection

In [6]:
base_url = "https://guide.michelin.com/en/it/restaurants"
pages = 100
functions.scrape_michelin_restaurants(base_url, pages)

In [7]:
# Read and visualize the lines of the file
with open("michelin_restaurant_urls.txt", "r") as file:
    urls = file.readlines()
# Remove the empty spaces
urls = [url.strip() for url in urls]
print(f"Total number of URL collected: {len(urls)}")

In [9]:
# Remope the duplicates
unique_urls = list(set(urls))
print(f"Number of unique URL: {len(unique_urls)}")
# Filter the URLs that contains "/restaurant/"
restaurant_urls = [url for url in unique_urls if "/restaurant/" in url]
print(f"Number of valid URLs: {len(restaurant_urls)}")

In [10]:
with open("michelin_restaurant_urls_cleaned.txt", "w") as file:
    for url in restaurant_urls:
        file.write(url + "\n")

print("Cleaned file saved as 'michelin_restaurant_urls_cleaned.txt'")

In [11]:
# Legge e visualizza le prime righe del file
with open("michelin_restaurant_urls_cleaned.txt", "r") as file:
    urls = file.readlines()
    
# Rimuove gli spazi vuoti e visualizza un esempio di URL raccolti
urls = [url.strip() for url in urls]
print(f"Total number of URL collected: {len(urls)}")

In [12]:
# Path with all the URLs
file_path = "C:/Users/xavie/Downloads/michelin_restaurant_urls_cleaned.txt"

functions.process_urls(file_path)

In [36]:
import glob
# Directory di base contenente tutte le cartelle di download
base_dir = 'C:/Users/xavie/Documents/ADM_HW3'

# Lista per memorizzare i dati di ogni ristorante
restaurants_data = []

# Iterazione su tutte le cartelle page_1, page_2, ..., page_100
for page_num in range(1, 101):
    folder_path = os.path.join(base_dir, f"page_{page_num}")
    
    # Trova tutti i file di testo che iniziano con "html_" nella cartella corrente
    html_files = glob.glob(os.path.join(folder_path, "html_*.txt"))
    print(html_files)
    
    for file_path in html_files:
        restaurant_info = functions.extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Creazione del DataFrame Pandas
df = pd.DataFrame(restaurants_data)

output_file_path = "C:/Users/xavie/Documents/ADM_HW3/restaurants_data.tsv"
df.to_csv(output_file_path, sep='\t', index=False)
print(f"Data has been saved in {output_file_path}")

Create TXT with urls of each restaurant page (just let it run, usually 2 minutes to finish)

In [3]:
import requests
from bs4 import BeautifulSoup

# URL di partenza
base_url = 'https://guide.michelin.com/en/it/restaurants/page/'

def scrape_restaurant_links():
    print("I'm starting to Scrape!")
    page = 1
    all_links = []

    while True:
        # Costruisci l'URL della pagina corrente
        url = f"{base_url}{page}"
        response = requests.get(url)

        # Verifica che la richiesta sia andata a buon fine
        if response.status_code != 200:
            print(f"Errore nel caricamento della pagina {page}")
            break

        # Parsing della pagina HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trova tutti i div con la classe specificata
        for class1_div in soup.select("div.card__menu-content.card__menu-content--flex.js-match-height-content"):
            # Cerca il tag <h3> con classe specificata e il tag <a> figlio
            h3 = class1_div.select_one("h3.card__menu-content--title.pl-text.pl-big.js-match-height-title a")
            if h3:
                link = h3.get("href")
                full_link = "https://guide.michelin.com" + link if link else None
                if full_link:
                    all_links.append(full_link)

        # Trova la sezione di paginazione
        pagination_lis = soup.select("div.js-restaurant__bottom-pagination ul li")

        # Trova l'elemento <li> con la classe "active"
        active_index = None
        for i, li in enumerate(pagination_lis):
            if li.select_one("a.active"):
                active_index = i
                break

        # Se c'è una pagina successiva, incrementa il numero di pagina
        if active_index is not None and active_index + 1 < len(pagination_lis):
            next_page = pagination_lis[active_index + 1].select_one("a")
            if next_page and next_page.get("href"):
                page += 1
            else:
                break
        else:
            break

    # Salva tutti i link dei ristoranti in un file
    with open("soupUrls.txt", "w") as file:
        for link in all_links:
            file.write(link + "\n")

    print(f"Scraping completed. {len(all_links)} link saved in soupUrls.txt.")

# Avvia lo scraping
scrape_restaurant_links()

Download each HTML page using .txt file just created.

In [4]:
#Scaricare html content da url

import aiohttp
import asyncio
import aiofiles
import os

CONCURRENT_REQUESTS = 20  # Lowered to reduce load on the server

async def load_urls(file_path):
    async with aiofiles.open(file_path, 'r') as f:
        urls = [line.strip() for line in await f.readlines()]
    return urls

async def download_url(session, url, output_dir):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
        'Referer': 'https://guide.michelin.com/',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    try:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                content = await response.text()
                filename = f"{output_dir}/{hash(url)}.html"
                async with aiofiles.open(filename, 'w') as f:
                    await f.write(content)
                print(f"Downloaded: {url}")
            else:
                print(f"Failed to download {url}: Status {response.status}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

async def download_all(urls, output_dir):
    connector = aiohttp.TCPConnector(limit=CONCURRENT_REQUESTS)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [download_url(session, url, output_dir) for url in urls]
        await asyncio.gather(*tasks)

if __name__ == "__main__":
    file_path = 'soupUrls.txt'
    output_dir = 'downloads'
    os.makedirs(output_dir, exist_ok=True)

    urls = await load_urls(file_path)
    await download_all(urls, output_dir)

Scrape each HTML page and create dataframe from data.

In [5]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup

# Directory containing the downloaded HTML files
output_dir = 'downloads'

# List to store restaurant data
restaurants_data = []

# Function to extract restaurant information from HTML
def extract_restaurant_info(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract information using CSS selectors or HTML structure of the page
        restaurant_info = {}

        # Ricerca div contenente info principali
        restaurantDetailsDiv = soup.find("div", class_="restaurant-details__components")

        # Ottiene tutte le row contenenti: Nome del ristorante (row1),
        # Indirizzo, prezzo e tipo cucina (row2), row 3 da scartare
        mainInfo = restaurantDetailsDiv.select("div.data-sheet > div.row")

        if mainInfo[0]:
            restaurant_info['restaurantName'] = mainInfo[0].find("h1", class_="data-sheet__title").text
        if mainInfo[1]:
            indirizzo_price = mainInfo[1].select("div.data-sheet__block > div.data-sheet__block--text")

            # Splitta la stringa contenente indirizzo, citta, CAP e nazione
            indirizzoList = indirizzo_price[0].text.strip().split(",")

            # Seleziona gli ultimi tre e li assegna a country, postalCode e city, tutto il resto verrà assegnato ad indirizzo
            restaurant_info['city'] = indirizzoList[-3]
            restaurant_info['postalCode'] = indirizzoList[-2]
            restaurant_info['country'] = indirizzoList[-1]
            restaurant_info['address'] = " ".join(indirizzoList[:-3]).strip().replace("\n", "") # Unisce tutti gli elementi precedenti agli ultimi tre

            # Split della riga contenente price e cuisineType info
            restaurant_info['priceRange'], restaurant_info['cuisineType'] = indirizzo_price[1].text.strip().split("·")

            restaurant_info['priceRange'] = restaurant_info['priceRange'].strip()
            # Possibili multiple cuisineType, dividi in lista
            restaurant_info['cuisineType'] = restaurant_info['cuisineType'].strip().split(",")

        # Description
        restaurant_info['description'] = soup.find("div", class_="data-sheet__description").text.strip().replace("\n", "")

        # Facilities and Services
        facilities = soup.select("div.restaurant-details__services ul li")
        restaurant_info['facilitiesServices'] = [s.text.strip() for s in facilities]

        # Accepted Credit Cards
        credit_cards = soup.select("div.list--card img")
        restaurant_info['creditCards'] = [re.search(r"(?<=\/)[a-z]*(?=-)", c.get("data-src"))[0] for c in credit_cards]

        # Phone Number
        spansDetails = restaurantDetailsDiv.select("section.section.section-main.section__text-componets.section__text-separator div.collapse__block-title div.d-flex span")
        restaurant_info['phoneNumber'] = spansDetails[0].text.strip()

        # URL
        restaurant_info['website'] = soup.find("meta", property="og:url")["content"]

    return restaurant_info

# Loop through all files in the directory and extract information
for filename in os.listdir(output_dir):
    if filename.endswith(".html"):
        print(filename)
        file_path = os.path.join(output_dir, filename)
        restaurant_info = extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Create a pandas DataFrame
df = pd.DataFrame(restaurants_data)

# Save the data to a CSV file
df.to_csv("restaurants_data.tsv", sep='\t', index=False)
print("Data saved to restaurants_data.csv")

# 2. Search Engine

## 2.0 Preprocessing

The first step is to preprocess the restaurant descriptions. For this, we use the custom-made function ```preprocessing```, and save all pre-processed documents in a list of documents ```preprocessed_docs```.

In [2]:
df = pd.read_csv('restaurants_data.tsv', sep = '\t')

In [3]:
df.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com


In [4]:
preprocessed_docs = defaultdict(list) # initialize defaultdict to store preprocessed docs
for doc_id, doc in enumerate(df.description):
  preprocessed_docs[doc_id] = functions.preprocessing(doc) # preprocess doc at position doc_id

In [5]:
# Test description
text = '''After many years' experience in Michelin-starred restaurants, Luigi Tramontano and his wife Nicoletta
have opened their first restaurant in the chef's native Gargnano. Previously a pasta factory, the building has been converted
into an elegant, contemporary-style restaurant which has nonetheless retained its charming high ceilings.
The cuisine is inspired by regional traditions which are reinterpreted to create gourmet dishes,
all prepared with respect for the ingredients used and a strong focus on local produce.'''

# Test preprocessing on test description
print(functions.preprocessing(text))

['mani', 'year', 'experi', 'michelin', 'star', 'restaur', 'luigi', 'tramontano', 'wife', 'nicoletta', 'open', 'first', 'restaur', 'chef', 'nativ', 'gargnano', 'previous', 'pasta', 'factori', 'build', 'convert', 'eleg', 'contemporary', 'styl', 'restaur', 'nonetheless', 'retain', 'charm', 'high', 'ceil', 'cuisin', 'inspir', 'region', 'tradit', 'reinterpret', 'creat', 'gourmet', 'dish', 'prepar', 'respect', 'ingredi', 'use', 'strong', 'focu', 'local', 'produc']


## 2.1 Conjunctive Query

### 2.1.1 Create your Index!

In the following code cells, we preprocess all restaurant descriptions and 1. save unique tokens in a DataFrame ```vocabulary_df``` that maps terms to unique integer IDs, then 2. compute the inverted index for the documents.

In [6]:
# 1. Vocabulary File

# Retrieve the restaurants DataFrame
df = pd.read_csv('restaurants_data.tsv', sep='\t')

doc_tokens = [] # initialize list to store all tokens

# Find unique tokens
for doc in preprocessed_docs.values():
  doc_tokens.extend(doc)
  doc_tokens = list(set(doc_tokens)) # remove duplicates

vocabulary_dict = {term: i for i,term in enumerate(doc_tokens)} # dictionary of all vocabulary terms
vocabulary_df = pd.DataFrame({'term': vocabulary_dict.keys(), 'term_id': vocabulary_dict.values()}) # dataframe that maps terms to IDs

vocabulary_df.to_csv('vocabulary.csv', index=False) # save vocabulary dataframe in a csv file

In [7]:
# 2. Inverted Index

inverted_index = defaultdict(list) # initialize inverted_index dictionary

# Compute the inverted_index
for doc_id, row in enumerate(df.description):
  tokens = set(preprocessed_docs[doc_id]) # preprocessed description
  for token in tokens: # eliminate duplicates
    # Look up the term_id of the current term/token
    term_id = vocabulary_dict[token]
    # If the doc_id is not in the term_id's list in inverted_index, add it
    if doc_id not in inverted_index[term_id]:
      inverted_index[term_id].append(doc_id)

# Save the inverted_index dictionary to a file
with open("inverted_index.pkl", "wb") as file:
    pickle.dump(inverted_index, file)

Next, we allow the user to input a query. After clicking on search, the first search engine will be triggered to retrieve all restaurants that contain in their description the same terms as the query.

In [8]:
import ipywidgets as widgets
from IPython.display import display
# re-load inverted index in case it was modified somewhere
with open('inverted_index.pkl', 'rb') as file:
    inverted_index = pickle.load(file)

# Text input field for query
text_input = widgets.Text(
    value='',
    placeholder='Type your query',
    description='Query:',
    disabled=False
)

# Search button
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='primary'
)

output = widgets.Output()

# Define a function to handle button press
def on_search_button_clicked(b):
    with output:
        output.clear_output()  # clear previous output if there are any
        query = text_input.value
        if query.strip():  # Check if there's an input
            display(functions.find_restaurants(query, vocabulary_df, inverted_index, df)) # display query results
        else:
            print("Please enter something to search for.")

# Link the function to the button
search_button.on_click(on_search_button_clicked)

# Display the widgets
display(text_input, search_button, output)

Text(value='', description='Query:', placeholder='Type your query')

Button(button_style='primary', description='Search', style=ButtonStyle())

Output()

## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

### 2.2.1 Inverted Index with TF-IDF Scores

In the following exercise, we will first compute the inverted index with TF-IDF scores using the custom-made function ```tf_idf``` and save the ```updated_inverted_index``` in a pickle file.

In [9]:
# Preliminary steps
n = len(preprocessed_docs)
updated_inverted_index = defaultdict(list) # initialize default dictionary to store the inverted_index values with TF-IDF scores
inverted_index_copy = inverted_index.copy() # Create a copy of the inverted_index to iterate over

# Compute updated_inverted_index
for term_id, docs in inverted_index_copy.items():
  tf_idf_scores = functions.tf_idf(int(term_id), inverted_index, preprocessed_docs, vocabulary_df, n)
  updated_inverted_index[term_id] = list(zip(docs, tf_idf_scores))

with open('updated_inverted_index.pkl', 'wb') as file:
    pickle.dump(updated_inverted_index, file)

Next, we retrieve from ```updated_inverted_index``` the TF-IDF scores related to documents, and memorize only the tuples (term, tf-idf) where tf-idf != 0 for each document in a pickle file.

In [10]:
# Compute the TF-IDF vectors of all documents and store them in a pickle file
doc_tf_idf_scores = defaultdict(list) # initialize dictionary to store non-zero TF-IDF scores for each document

for term_id, docs_scores in updated_inverted_index.items():
  for doc_id, tf_idf_score in docs_scores:
    if tf_idf_score != 0:
      doc_tf_idf_scores[doc_id].append((term_id,tf_idf_score))
  doc_tf_idf_scores[doc_id].sort(key=lambda x: x[0]) # sort the terms

with open('doc_tf_idf_scores.pkl', 'wb') as file:
    pickle.dump(doc_tf_idf_scores, file)

Finally, we enable the user to input a text query, and return the top-k ranked restaurants by cosine similarity.

In [12]:
# re-load inverted index in case it was modified somewhere
with open('inverted_index.pkl', 'rb') as file:
    inverted_index = pickle.load(file)

# Text input field for query
text_input = widgets.Text(
    value='',
    placeholder='Type your query',
    description='Query:',
    disabled=False
)

# Search button
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='primary'
)

output = widgets.Output()

# Define a function to handle button press
def on_search_button_clicked(b):
    with output:
        output.clear_output()  # clear previous output if there are any
        query = text_input.value
        if query.strip():  # Check if there's an input
            k = 10
            display(functions.top_k_restaurants(query, inverted_index, vocabulary_dict, doc_tf_idf_scores, df, k, n)) # display query results
        else:
            print("Please enter something to search for.")

# Link the function to the button
search_button.on_click(on_search_button_clicked)

# Display the widgets
display(text_input, search_button, output)

Text(value='', description='Query:', placeholder='Type your query')

Button(button_style='primary', description='Search', style=ButtonStyle())

Output()

## 3. Define a New Score!

A user interface(UI) is used to consider the user's additive information, which allows the user to enter the fields they prefer to find the restaurant.
We will use this additional information to predicate some of the rows in our dataset.
Since all restaurants are filtered by text initially, we want to give lower weight to the similarity between the description and the input query.  
Rather we want to give higher weight to user input details, which are the key to finding restaurants to which a user can bring interest.
Specifically we prefer to give higher weight to the type of cuisine, if I were to search for a restaurant and I have a particular desire to eat that cuisine I would want those as the pirmiest search results. Sucessively we give slightly less weight to the services available and finally the least weight to the price, speaking of Michelin restaurants the price is not an issue
We can also find this behavior in very popular search engines, for example amazon search, prefers user-selected filters over the user's even detailed content in the search bar.

The results in the end turn out to be better, more accurate. We also have to consider that in the previous Engine based on the cosine similarity we are just considering the score based on the similarity between the input query and the description which is way larger than a possible query. Instead now we are building our own custom score where we give more importance to the details, infact we havebigger score for the ones that perfectly fit thee search. We are basically considernig more fileds so its obvious that the results are more accurated. But there is also to note that at the computational level this type of search is less performant since we have to initially load all the restaurants that respect the query input and then create a new score based on matching the user input and finally sort to select only the first k.

In [13]:
from search_restaurant_ui import SearchRestaurantUI

In [14]:
search_ui = SearchRestaurantUI(df, vocabulary_df, inverted_index)
search_ui.display()

## 4. Visualizing the Most Relevant Restaurants

### 4.1 Geocode Locations

In 4.1 we use geopy to get the coordinates of the restaurants and save them in the .tsv file. 

If some data of these restaurants is not found using geopy, their name, postal code and country are saved in a second file "noDataRest.tsv" to be used in any (possible) subsequent processing to recover the missing data of these restaurants.

In [15]:
import os
import pandas as pd
import json
from bs4 import BeautifulSoup
import glob

# Directory where .html files are stored
output_dir = 'C:/Users/xavie/Documents/ADM_HW3'

# List to store restaurant data
restaurants_data = []

# Function to extract restaurant information from HTML
def extract_restaurant_info(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract information using HTML structure of the page
        restaurant_info = {}

        script_tag = soup.find("script", type="application/ld+json")

        if script_tag:
            json_content = json.loads(script_tag.string)

            restaurant_info['restaurantName'] = json_content['name']
            restaurant_info['region'] = json_content['address']['addressRegion']
            restaurant_info['latitude'] = json_content['latitude']
            restaurant_info['longitude'] = json_content['longitude']

    return restaurant_info

# Loop through all files in the directory and extract information
# Itera attraverso le pagine da 1 a 100
for page_num in range(1, 101):
    # Costruisci il percorso della cartella per ciascuna pagina
    folder_path = os.path.join(output_dir, f"page_{page_num}")
    
    # Trova tutti i file di testo che iniziano con "html_" nella cartella corrente
    html_files = glob.glob(os.path.join(folder_path, "html_*.txt"))
    
    # Itera attraverso i file trovati e processa ciascuno
    for file_path in html_files:
        print(file_path)
        restaurant_info = extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Create a pandas DataFrame
df = pd.DataFrame(restaurants_data)

# Translation of italian region from english to italian
translations = {
    "Aosta Valley": "Valle d'Aosta/Vallée d'Aoste",
    "Piedmont": "Piemonte",
    "Lombardy": "Lombardia",
    "Sicily": "Sicilia",
    "Tuscany": "Toscana",
    "Apulia": "Puglia",
    "Trentino-South Tyrol": "Trentino-Alto Adige/Südtirol",
    "Sardinia": "Sardegna"
}
df['region'] = df['region'].replace(translations) 

# Save the data to a CSV file
df.to_csv("geodata.tsv", sep='\t', index=False)
print("Data saved to restaurants_data.csv")

C:/Users/xavie/Documents/ADM_HW3\page_1\html_arca.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_cannavacciuolo-le-cattedrali-asti.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_capriccio.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_cestello-ristoclub.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_corte-sconta.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_del-belbo-da-bardon.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_essentia.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_feria.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_filippino.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_gimmi-restaurant.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_la-coccinella.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_la-ferrata.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_la-gioconda.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_la-madia.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_la-torre153575.txt
C:/Users/xavie/Documents/ADM_HW3\page_1\html_locanda-marchesa

KeyboardInterrupt: 

In [16]:
df = pd.read_csv("restaurants_data.tsv", sep='\t')

In [17]:
df.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com


In [18]:
df['restaurantId'] = range(1, len(df) + 1)

df.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,restaurantId
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/,1
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/,2
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/,3
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN,4
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com,5


In [19]:
df.shape

(2006, 13)

In [20]:
df_geo = pd.read_csv("geodata.tsv", sep='\t')
df_geo['restaurantId'] = range(1, len(df) + 1)
df_geo.head()

,restaurantName,region,latitude,longitude,restaurantId
0,Arca,Marche,42.926408,13.897645,1
1,Cannavacciuolo Le Cattedrali Asti,Piemonte,44.907837,8.117304,2
2,Capriccio,Lombardia,45.552850,10.561210,3
3,Cestello Firenze,Toscana,43.770306,11.243657,4
4,Corte Sconta,Veneto,45.434791,12.347919,5


In [21]:
df_geo.shape

(2006, 5)

In [22]:
df_finale = pd.merge(df, df_geo, on='restaurantId')

# Verifica le dimensioni e le prime righe
print(df_finale.shape)  # Questo dovrebbe restituire (1962, 15)

(2006, 17)


In [23]:
df_finale = df_finale.rename(columns={'restaurantName_x': 'restaurantName'})

# Droppa la colonna 'restaurantName_y'
df_finale = df_finale.drop(columns=['restaurantName_y'])

In [24]:
df_finale.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,restaurantId,region,latitude,longitude
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/,1,Marche,42.926408,13.897645
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/,2,Piemonte,44.907837,8.117304
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/,3,Lombardia,45.552850,10.561210
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN,4,Toscana,43.770306,11.243657
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com,5,Veneto,45.434791,12.347919


In [25]:
df_finale.shape

(2006, 16)

### 4.3 Map Setup

Dash used instead of plain Plotly to make the map interactive and to be able to select regions with a simple click instead of using a list.

In [26]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import geopandas as gpd
import pandas as pd
import json

# DEBUG
print(df_finale.shape)

# Load GeoJSON file of Italian regions in a GeoDataFrame
gdf = gpd.read_file("./GeoJson/limits_IT_regions.geojson")

# Check CRS and convert it if needed (important to calculate centroid of each region to display it centered)
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Extract region names from the GeoDataFrame
region_names = gdf['reg_name'].tolist()

# Create Dash app (Dash provides an easy-to-use API for creating web apps using Python)
app = dash.Dash(__name__)

# Layout of the app with checkbox for filtering top_k_result.tsv
app.layout = html.Div([
    html.Div([
        # Add checkbox to filter top_k_result.tsv
        dcc.Checklist(
            id='show-top-k',
            options=[{'label': 'Show Top-K Restaurants Only', 'value': 'top_k'}],
            value=[],
            inline=True
        ),
    ]),
    
    # Create 2 maps, one of Italy with selectable region and one for the selected region where we display restaurants
    html.Div([
        dcc.Graph(id='italy-map', clickData=None, style={'width': '50vw', 'height': '100vh'}),
        dcc.Graph(id='region-map', style={'width': '50vw', 'height': '100vh'})
    ], style={'display': 'flex', 'flex-direction': 'row'})
])

# Global variable to store filtered restaurants (default is all restaurants)
filtered_restaurants = df_finale

@app.callback(
    [Output('italy-map', 'figure'),
     Output('region-map', 'figure')],
    [Input('italy-map', 'clickData'),
     Input('show-top-k', 'value')]
)
def update_maps(clickData, show_top_k):
    global filtered_restaurants
    
    # Se la checkbox "Top-K" è selezionata, filtra i ristoranti dal file top_k_result.tsv
    if 'top_k' in show_top_k:
        top_k_df = pd.read_csv('top_k_result.tsv', sep='\t')
        filtered_restaurants = df_finale[df_finale['restaurantName'].isin(top_k_df['restaurantName'])]
    else:
        filtered_restaurants = df_finale

    # Variabile per gestire se una regione è selezionata
    selected_region = None
    if clickData:
        selected_region = clickData['points'][0]['location']

    # Configurazione della mappa dell'Italia
    italy_map = px.choropleth_mapbox(
        geojson=json.loads(gdf.to_json()),
        locations=gdf['reg_name'],
        featureidkey="properties.reg_name",
        color_continuous_scale=['grey'],
        opacity=0.2,
        mapbox_style="carto-positron",
        zoom=4.5,
        center={"lat": 41.8719, "lon": 12.5674},
        title="Map of the Italian Regions"
    )
    
    # Se la checkbox "Top-K" è selezionata e una regione è selezionata, mostra i marker
    if 'top_k' in show_top_k and selected_region:
        for price_range, color in [('€', 'blue'), ('€€', 'green'), ('€€€', 'purple'), ('€€€€', 'brown')]:
            price_restaurants = filtered_restaurants[filtered_restaurants['priceRange'] == price_range]
            italy_map.add_scattermapbox(
                lat=price_restaurants['latitude'].tolist(),
                lon=price_restaurants['longitude'].tolist(),
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.8),
                text=price_restaurants['restaurantName'],
                name=price_range,
                hovertemplate=(
                    "<b>Restaurant Name:</b> %{text}<br>" +
                    "<b>Latitude:</b> %{lat}<br>" +
                    "<b>Longitude:</b> %{lon}<br>"
                )
            )
    
    if selected_region:
        italy_map.update_traces(marker_line_color="red", selector=dict(location=selected_region))
    
    # Configurazione della mappa della regione selezionata (se presente)
    if selected_region:
        region_gdf = gdf[gdf['reg_name'] == selected_region]
        region_gdf_proj = region_gdf.to_crs(epsg=32632)
        centroid_proj = region_gdf_proj.geometry.centroid.iloc[0]
        centroid = gpd.GeoSeries([centroid_proj], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
        center_lat, center_lon = centroid.y, centroid.x

        filtered_restaurants_region = filtered_restaurants[filtered_restaurants['region'] == selected_region]

        region_map = px.choropleth_mapbox(
            geojson=json.loads(region_gdf.to_json()),
            locations=[selected_region],
            featureidkey="properties.reg_name",
            mapbox_style="carto-positron",
            color_discrete_sequence=['red'],
            opacity=0.1,
            zoom=6,
            center={"lat": center_lat, "lon": center_lon},
            title=f"Map of {selected_region} with Restaurants"
        )
        
        # Aggiungi marker per i ristoranti nella regione selezionata
        for price_range, color in [('€', 'blue'), ('€€', 'green'), ('€€€', 'purple'), ('€€€€', 'brown')]:
            price_restaurants = filtered_restaurants_region[filtered_restaurants_region['priceRange'] == price_range]
            region_map.add_scattermapbox(
                lat=price_restaurants['latitude'].tolist(),
                lon=price_restaurants['longitude'].tolist(),
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.8),
                text=price_restaurants['restaurantName'],
                name=price_range,
                hovertemplate=(
                    "<b>Restaurant Name:</b> %{text}<br>" +
                    "<b>Latitude:</b> %{lat}<br>" +
                    "<b>Longitude:</b> %{lon}<br>"
                )
            )
    else:
        # Se nessuna regione è selezionata, restituisci una mappa vuota
        region_map = {}

    return italy_map, region_map

# Start Dash App
if __name__ == '__main__':
    app.run_server(jupyter_mode = 'external', debug=True)


(2006, 16)
Dash app running on http://127.0.0.1:8050/


## 5. BONUS: Advanced Search Engine

In [ ]:
# 1. Vocabulary File

doc_tokens = [] # initialize list to store all tokens

for idx, row in enumerate(df.description):
  doc_tokens.extend(functions.preprocessing(row))
  doc_tokens = list(set(doc_tokens)) # remove duplicates

vocabulary_dict = {term: i for i,term in enumerate(doc_tokens)}

vocabulary_df = pd.DataFrame({'term': vocabulary_dict.keys(),
                              'term_id': vocabulary_dict.values()})

vocabulary_df.to_csv('vocabulary.csv', index=False)

In [ ]:
# 2. Inverted Index

inverted_index = defaultdict(list) # initialize inverted_index dictionary
preprocessed_docs = defaultdict(list) # initialize dictionary to store pre-processed docs (restaurant descriptions)

for doc_id, row in enumerate(df.description):
  preprocessed_docs[doc_id] = functions.preprocessing(row) # preprocess the description
  tokens = set(preprocessed_docs[doc_id]) # preprocess the description
  for token in tokens: # eliminate duplicates
    # Look up the term_id of the current term/token
    term_id = vocabulary_dict[token]
    # If the doc_id is not in the term_id's list in inverted_index, add it
    if doc_id not in inverted_index[term_id]:
      inverted_index[term_id].append(doc_id)

In [ ]:
# @title Compute updated_inverted_index

n = len(preprocessed_docs)
updated_inverted_index = defaultdict(list) # initialize default dictionary to store the inverted_index values with TF-IDF scores

# Create a copy of the inverted_index to iterate over
inverted_index_copy = inverted_index.copy()

for term_id, docs in inverted_index_copy.items():
  tf_idf_scores = functions.tf_idf(int(term_id), inverted_index, preprocessed_docs, vocabulary_df, n)
  updated_inverted_index[term_id] = list(zip(docs, tf_idf_scores))

with open('updated_inverted_index.pkl', 'wb') as file:
    pickle.dump(updated_inverted_index, file)

In [ ]:
doc_tf_idf_scores = defaultdict(list) # initialize dictionary to store non-zero TF-IDF scores for each document

for term_id, docs_scores in updated_inverted_index.items():
  for doc_id, tf_idf_score in docs_scores:
    if tf_idf_score != 0:
      doc_tf_idf_scores[doc_id].append((term_id,tf_idf_score))
  doc_tf_idf_scores[doc_id].sort(key=lambda x: x[0]) # sort the terms

with open('doc_tf_idf_scores.pkl', 'wb') as file:
    pickle.dump(doc_tf_idf_scores, file)

In [ ]:
  processed_query = functions.preprocessing(query) # processed query
  query_tokens = list(set(processed_query)) # unique query tokens
  # print(query_tokens) # debugging line
  # Find all docs to consider
  docs_to_consider = [] # initialize list to store documents to consider (non-zero intersection with the query tokens)

  for token in query_tokens:
    if vocabulary_dict[token]: # check if the token is in the vocabulary
      token_id = vocabulary_dict[token] # get the term_id of the token
      docs_to_consider.extend(inverted_index[token_id]) # add the documents that contain the token to the docs to consider

  docs_to_consider = list(set(docs_to_consider))
   # Calculate the TF-IDF score of the query
  query_tf_idf_scores = [] # initialize list to store the TF-IDF scores of the query
  for term in query_tokens:
    term_id = vocabulary_dict[term] # get the term_id of the term
    #print(inverted_index[term_id]) # debugging line
    n_term = len(inverted_index[term_id]) # number of documents that contain the term
    IDF = np.log10(n / n_term) # calculate IDF of the term
    TF = processed_query.count(term) # calculate TF of the term
    #print(f"TF = {TF}") # debugging line
    #print(f"IDF = {IDF}") # debuggin line
    query_tf_idf_scores.append((term_id, TF * IDF)) # calculate TF-IDF score

  query_tf_idf_scores.sort(key=lambda x: x[0]) # sort the query_tf_idf_scores in order of term_id

  query_norm = np.linalg.norm(np.array([score for _, score in query_tf_idf_scores])) # calculate the norm of the query
  #print(f"query tf_idf_scores: {query_tf_idf_scores}") # debuggin line
  #print(f"query norm: {query_norm}") # debugging line
  # calculate document norms
  doc_norms = {doc_id: np.linalg.norm(np.array([doc_tf_idf_scores[doc_id][i][1] for i in range(len(doc_tf_idf_scores[doc_id]))])) for doc_id in docs_to_consider}

  # Function that returns two lists of tuples (term, query_tf_idf) and (term, doc_tf_idf) such that
  # the terms are in the intersection of the query terms and the doc's terms
  def query_doc_intersection(query_tf_idf_scores, doc_tf_idf_scores):
    '''
    Calculate the intersection of the query and the document
    Inputs:
    query_terms: list of sorted unique query terms
    doc_terms: list of sorted unique document terms
    Output:
    query_intersection: list of tuples (term, query_tf_idf)
    doc_intersection: list of tuples (term, doc_tf_idf)
    '''
    query_intersection = [] # initialize list to store (term, query_tf_idf) tuples in the intersection
    doc_intersection = [] # initialize list to store (term, doc_tf_idf) tuples in the intersection
    i, j = 0, 0 # initialize two pointers
    while i<len(query_tf_idf_scores) and j<len(doc_tf_idf_scores):
      if query_tf_idf_scores[i][0] == doc_tf_idf_scores[j][0]:
        query_intersection.append(query_tf_idf_scores[i])
        doc_intersection.append(doc_tf_idf_scores[j])
        i += 1
        j += 1
      elif query_tf_idf_scores[i][0] < doc_tf_idf_scores[j][0]:
        i += 1
      else:
        j += 1
    return query_intersection, doc_intersection

  # Calculate cosine-similarity between the query and each document
  cosine_similarity = defaultdict(float) # initialize dictionary to store the cosine similarity results
  for doc_id in docs_to_consider:
    query_intersection, doc_intersection = query_doc_intersection(query_tf_idf_scores, doc_tf_idf_scores[doc_id]) # find the
    cosine_similarity[doc_id] = np.dot(np.array([score for _, score in query_intersection]), np.array([score for _, score in doc_intersection])) / (query_norm * doc_norms[doc_id])

In [39]:
from collections import defaultdict
import numpy as np
import pandas as pd
import pickle

# Assuming `df` has `restaurantName`, `city`, `cuisineType`, and `description` columns

# Helper function to create vocabulary and inverted index for a field
def build_vocabulary_inverted_index(field_data, preprocess_func):
    tokens = []
    for value in field_data:
        tokens.extend(preprocess_func(value))
    tokens = list(set(tokens))  # remove duplicates

    vocabulary_dict = {term: i for i, term in enumerate(tokens)}
    inverted_index = defaultdict(list)

    for doc_id, value in enumerate(field_data):
        processed_tokens = set(preprocess_func(value))
        for token in processed_tokens:
            term_id = vocabulary_dict[token]
            inverted_index[term_id].append(doc_id)

    return vocabulary_dict, inverted_index

# Build vocabulary and inverted indexes for each field
vocab_restaurantName, inverted_index_restaurantName = build_vocabulary_inverted_index(df['restaurantName'], functions.preprocessing)
vocab_city, inverted_index_city = build_vocabulary_inverted_index(df['city'], functions.preprocessing)
vocab_cuisineType, inverted_index_cuisineType = build_vocabulary_inverted_index(df['cuisineType'], functions.preprocessing)

# Convert each vocabulary dictionary to a DataFrame
vocabulary_df_restaurantName = pd.DataFrame({'term': vocab_restaurantName.keys(), 'term_id': vocab_restaurantName.values()})
vocabulary_df_city = pd.DataFrame({'term': vocab_city.keys(), 'term_id': vocab_city.values()})
vocabulary_df_cuisineType = pd.DataFrame({'term': vocab_cuisineType.keys(), 'term_id': vocab_cuisineType.values()})

# Function to compute TF-IDF scores with correct vocabulary_df input
def compute_tf_idf_scores(inverted_index, preprocessed_docs, vocabulary_df, n_docs):
    updated_inverted_index = defaultdict(list)
    for term_id, docs in inverted_index.items():
        tf_idf_scores = functions.tf_idf(term_id, inverted_index, preprocessed_docs, vocabulary_df, n_docs)
        updated_inverted_index[term_id] = list(zip(docs, tf_idf_scores))
    return updated_inverted_index

n_docs = len(df)

# Process each field with the updated vocabulary DataFrames
preprocessed_restaurantName = {doc_id: functions.preprocessing(name) for doc_id, name in enumerate(df['restaurantName'])}
updated_inverted_index_restaurantName = compute_tf_idf_scores(inverted_index_restaurantName, preprocessed_restaurantName, vocabulary_df_restaurantName, n_docs)

preprocessed_city = {doc_id: functions.preprocessing(city) for doc_id, city in enumerate(df['city'])}
updated_inverted_index_city = compute_tf_idf_scores(inverted_index_city, preprocessed_city, vocabulary_df_city, n_docs)

preprocessed_cuisineType = {doc_id: functions.preprocessing(cuisine) for doc_id, cuisine in enumerate(df['cuisineType'])}
updated_inverted_index_cuisineType = compute_tf_idf_scores(inverted_index_cuisineType, preprocessed_cuisineType, vocabulary_df_cuisineType, n_docs)


# 3. Aggregate Cosine Similarity Scores Across Fields
def compute_cosine_similarity(query_tf_idf_scores, doc_tf_idf_scores, query_norm, doc_norms, weight=1.0):
    cosine_similarity = defaultdict(float)
    for doc_id in doc_tf_idf_scores:
        query_intersection, doc_intersection = query_doc_intersection(query_tf_idf_scores, doc_tf_idf_scores[doc_id])
        score = np.dot(np.array([score for _, score in query_intersection]), np.array([score for _, score in doc_intersection]))
        cosine_similarity[doc_id] += weight * score / (query_norm * doc_norms[doc_id]) if doc_norms[doc_id] != 0 else 0
    return cosine_similarity

# 4. Calculate aggregate similarity scores for all fields
def aggregate_similarity(query_tokens, inverted_indices, vocabularies, doc_tf_idf_scores_dict, n_docs, weights):
    cosine_scores = defaultdict(float)
    for field, inverted_index in inverted_indices.items():
        vocabulary = vocabularies[field]
        updated_inverted_index = doc_tf_idf_scores_dict[field]
        query_tf_idf_scores = []

        for token in query_tokens:
            if token in vocabulary:
                term_id = vocabulary[token]
                query_tf_idf_scores.append((term_id, functions.tf_idf(term_id, inverted_index, {0: query_tokens}, vocabulary, n_docs)[0]))

        query_tf_idf_scores.sort(key=lambda x: x[0])
        query_norm = np.linalg.norm([score for _, score in query_tf_idf_scores])
        
        field_cosine_scores = compute_cosine_similarity(query_tf_idf_scores, updated_inverted_index, query_norm, weights[field])
        for doc_id, score in field_cosine_scores.items():
            cosine_scores[doc_id] += score

    return cosine_scores


In [40]:
query = "pizza New York cheap"  # Example query
processed_query = functions.preprocessing(query)
inverted_indices = {
    "restaurantName": updated_inverted_index_restaurantName,
    "city": updated_inverted_index_city,
    "cuisineType": updated_inverted_index_cuisineType
}
vocabularies = {
    "restaurantName": vocab_restaurantName,
    "city": vocab_city,
    "cuisineType": vocab_cuisineType
}
doc_tf_idf_scores_dict = {
    "restaurantName": updated_inverted_index_restaurantName,
    "city": updated_inverted_index_city,
    "cuisineType": updated_inverted_index_cuisineType
}

# Weights for each field
weights = {"restaurantName": 0.5, "city": 0.3, "cuisineType": 0.2}

# Aggregate similarity scores across all fields
cosine_scores = aggregate_similarity(processed_query, inverted_indices, vocabularies, doc_tf_idf_scores_dict, n_docs, weights)


In [14]:
import pandas as pd
from collections import defaultdict
import numpy as np
import pickle

# Assuming `df` has `restaurantName`, `city`, `cuisineType` columns
# Assuming `functions` module has a `preprocessing` function and a `tf_idf` function

# Helper function to create vocabulary and inverted index for a field
def build_vocabulary_inverted_index(field_data, preprocess_func):
    tokens = []
    for value in field_data:
        tokens.extend(preprocess_func(value))
    tokens = list(set(tokens))  # Remove duplicates

    vocabulary_dict = {term: i for i, term in enumerate(tokens)}
    inverted_index = defaultdict(list)

    for doc_id, value in enumerate(field_data):
        processed_tokens = set(preprocess_func(value))
        for token in processed_tokens:
            term_id = vocabulary_dict[token]
            inverted_index[term_id].append(doc_id)

    return vocabulary_dict, inverted_index

# Build vocabulary and inverted indexes for each field
vocab_restaurantName, inverted_index_restaurantName = build_vocabulary_inverted_index(df['restaurantName'], functions.preprocessing)
vocab_city, inverted_index_city = build_vocabulary_inverted_index(df['city'], functions.preprocessing)
vocab_cuisineType, inverted_index_cuisineType = build_vocabulary_inverted_index(df['cuisineType'], functions.preprocessing)

# Convert each vocabulary dictionary to a DataFrame for TF-IDF calculations
vocabulary_df_restaurantName = pd.DataFrame({'term': vocab_restaurantName.keys(), 'term_id': vocab_restaurantName.values()})
vocabulary_df_city = pd.DataFrame({'term': vocab_city.keys(), 'term_id': vocab_city.values()})
vocabulary_df_cuisineType = pd.DataFrame({'term': vocab_cuisineType.keys(), 'term_id': vocab_cuisineType.values()})

# Function to compute TF-IDF scores with the appropriate vocabulary_df
def compute_tf_idf_scores(inverted_index, preprocessed_docs, vocabulary_df, n_docs):
    updated_inverted_index = defaultdict(list)
    for term_id, docs in inverted_index.items():
        tf_idf_scores = functions.tf_idf(term_id, inverted_index, preprocessed_docs, vocabulary_df, n_docs)
        updated_inverted_index[term_id] = list(zip(docs, tf_idf_scores))
    return updated_inverted_index

n_docs = len(df)

# Process each field with the updated vocabulary DataFrames
preprocessed_restaurantName = {doc_id: functions.preprocessing(name) for doc_id, name in enumerate(df['restaurantName'])}
updated_inverted_index_restaurantName = compute_tf_idf_scores(inverted_index_restaurantName, preprocessed_restaurantName, vocabulary_df_restaurantName, n_docs)

preprocessed_city = {doc_id: functions.preprocessing(city) for doc_id, city in enumerate(df['city'])}
updated_inverted_index_city = compute_tf_idf_scores(inverted_index_city, preprocessed_city, vocabulary_df_city, n_docs)

preprocessed_cuisineType = {doc_id: functions.preprocessing(cuisine) for doc_id, cuisine in enumerate(df['cuisineType'])}
updated_inverted_index_cuisineType = compute_tf_idf_scores(inverted_index_cuisineType, preprocessed_cuisineType, vocabulary_df_cuisineType, n_docs)

# Aggregate cosine similarity scores across restaurantName, city, and cuisineType fields
def aggregate_similarity(query, inverted_indices, vocabularies, updated_inverted_indices, n_docs, weights):
    processed_query = functions.preprocessing(query)
    cosine_scores = defaultdict(float)

    for field, inverted_index in inverted_indices.items():
        vocabulary = vocabularies[field]
        updated_inverted_index = updated_inverted_indices[field]

        # Calculate TF-IDF scores for query terms in this field
        query_tf_idf_scores = []
        for token in processed_query:
            if token in vocabulary:
                term_id = vocabulary[token]
                query_tf_idf_scores.append((term_id, functions.tf_idf(term_id, inverted_index, {0: processed_query}, vocabulary, n_docs)[0]))

        query_tf_idf_scores.sort(key=lambda x: x[0])
        query_norm = np.linalg.norm([score for _, score in query_tf_idf_scores])

        # Compute cosine similarity for each document in this field
        doc_norms = {
            doc_id: np.linalg.norm([score for _, score in updated_inverted_index[doc_id]])
            for doc_id in updated_inverted_index
        }

        for doc_id, doc_tf_idf_scores in updated_inverted_index.items():
            # Calculate intersection of query and document TF-IDF scores
            query_intersection, doc_intersection = functions.query_doc_intersection_enanched(query_tf_idf_scores, doc_tf_idf_scores)
            dot_product = np.dot([score for _, score in query_intersection], [score for _, score in doc_intersection])

            # Calculate and aggregate cosine similarity
            cosine_scores[doc_id] += weights[field] * (dot_product / (query_norm * doc_norms.get(doc_id, 1e-10)))

    return cosine_scores

# Define weights for each field
weights = {"restaurantName": 0.4, "city": 0.3, "cuisineType": 0.3}

# Query processing function
def search(query, weights):
    inverted_indices = {
        "restaurantName": inverted_index_restaurantName,
        "city": inverted_index_city,
        "cuisineType": inverted_index_cuisineType
    }
    vocabularies = {
        "restaurantName": vocab_restaurantName,
        "city": vocab_city,
        "cuisineType": vocab_cuisineType
    }
    updated_inverted_indices = {
        "restaurantName": updated_inverted_index_restaurantName,
        "city": updated_inverted_index_city,
        "cuisineType": updated_inverted_index_cuisineType
    }
    
    # Aggregate similarity scores across specified fields
    cosine_scores = aggregate_similarity(query, inverted_indices, vocabularies, updated_inverted_indices, n_docs, weights)
    
    # Sort results by similarity score
    ranked_docs = sorted(cosine_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_docs[:5]  # Return top 5 results

# Example query
query = "pizza New York Italian"
top_results = search(query, weights)
print(top_results)


In [22]:
import json
def createIndex(df, column):
    # Convert all unique words to lowercase and apply stemming
    stemmer = PorterStemmer()
    df[column] = df[column].fillna('')
    all_words = pd.Series(" ".join(df[column].str.lower()).split()).unique()
    all_words_stemmed = [stemmer.stem(word) for word in all_words]
    
    # Initialize the dictionary for stemmed words
    word_to_restaurants = {word: [] for word in all_words_stemmed}
    
    # Split each row's text, stem each word, and update the index
    for i, row in df.iterrows():
        words = row[column].lower().split()
        for word in words:
            stemmed_word = stemmer.stem(word)
            word_to_restaurants[stemmed_word].append(row['restaurantName'])
    
    # Save to JSON
    path = f'C:/Users/xavie/Documents/ADM_HW3/{column}_index.json'
    with open(path, 'w') as jsonfile:
        json.dump(word_to_restaurants, jsonfile)

In [23]:
restaurnt_index = createIndex(df, 'restaurantName')

In [26]:
city_index = createIndex(df, 'city')

In [27]:
cuisinType_index = createIndex(df, 'cuisineType')

In [4]:
# Initialize inverted indexes for each field
inverted_index_name = defaultdict(list)
inverted_index_city = defaultdict(list)
inverted_index_cuisine = defaultdict(list)

# Preprocessed documents and vocabulary for each field
preprocessed_docs_name = defaultdict(list)
preprocessed_docs_city = defaultdict(list)
preprocessed_docs_cuisine = defaultdict(list)

vocabulary_dict_name = {}
vocabulary_dict_city = {}
vocabulary_dict_cuisine = {}

# Total number of documents
n = len(df)

# Preprocess each field, create inverted index, and calculate TF-IDF
for doc_id, row in df.iterrows():

    # Restaurant name field
    preprocessed_docs_name[doc_id] = functions.preprocessing(row['restaurantName'])
    tokens_name = set(preprocessed_docs_name[doc_id])
    for token in tokens_name:
        # Update vocabulary and get term_id
        term_id = vocabulary_dict_name.setdefault(token, len(vocabulary_dict_name))
        # Calculate TF-IDF
        tf = preprocessed_docs_name[doc_id].count(token)
        df_name = len(inverted_index_name[term_id]) if term_id in inverted_index_name else 0
        idf = np.log10(n / (df_name + 1))  # Add 1 to avoid division by zero
        tf_idf_score = tf * idf
        # Append (doc_id, tf_idf_score) to inverted index
        inverted_index_name[term_id].append((doc_id, tf_idf_score))

    # City field
    preprocessed_docs_city[doc_id] = functions.preprocessing(row['city'])
    tokens_city = set(preprocessed_docs_city[doc_id])
    for token in tokens_city:
        term_id = vocabulary_dict_city.setdefault(token, len(vocabulary_dict_city))
        tf = preprocessed_docs_city[doc_id].count(token)
        df_city = len(inverted_index_city[term_id]) if term_id in inverted_index_city else 0
        idf = np.log10(n / (df_city + 1))
        tf_idf_score = tf * idf
        inverted_index_city[term_id].append((doc_id, tf_idf_score))

    # Cuisine type field
    preprocessed_docs_cuisine[doc_id] = functions.preprocessing(row['cuisineType'])
    tokens_cuisine = set(preprocessed_docs_cuisine[doc_id])
    for token in tokens_cuisine:
        term_id = vocabulary_dict_cuisine.setdefault(token, len(vocabulary_dict_cuisine))
        tf = preprocessed_docs_cuisine[doc_id].count(token)
        df_cuisine = len(inverted_index_cuisine[term_id]) if term_id in inverted_index_cuisine else 0
        idf = np.log10(n / (df_cuisine + 1))
        tf_idf_score = tf * idf
        inverted_index_cuisine[term_id].append((doc_id, tf_idf_score))

# Save inverted indexes to files if needed
with open("inverted_index_restaurantName.pkl", "wb") as file:
    pickle.dump(inverted_index_name, file)
# Repeat for other fields...
with open("inverted_index_city.pkl", "wb") as file:
    pickle.dump(inverted_index_city, file)
with open("inverted_index_cuisineType.pkl", "wb") as file:
    pickle.dump(inverted_index_cuisine, file)
    
# Esempio per calcolare doc_tf_idf_scores per 'restaurantName'
doc_tf_idf_scores_name = defaultdict(list)
for term_id, docs_scores in inverted_index_name.items():  # usa l'indice invertito specifico per 'restaurantName'
    for doc_id, tf_idf_score in docs_scores:
        if tf_idf_score != 0:
            doc_tf_idf_scores_name[doc_id].append((term_id, tf_idf_score))
    doc_tf_idf_scores_name[doc_id].sort(key=lambda x: x[0])  # ordina per term_id

# Ripeti per 'city' e 'cuisineType'
doc_tf_idf_scores_city = defaultdict(list)
for term_id, docs_scores in inverted_index_city.items():
    for doc_id, tf_idf_score in docs_scores:
        if tf_idf_score != 0:
            doc_tf_idf_scores_city[doc_id].append((term_id, tf_idf_score))
    doc_tf_idf_scores_city[doc_id].sort(key=lambda x: x[0])

doc_tf_idf_scores_cuisine = defaultdict(list)
for term_id, docs_scores in inverted_index_cuisine.items():
    for doc_id, tf_idf_score in docs_scores:
        if tf_idf_score != 0:
            doc_tf_idf_scores_cuisine[doc_id].append((term_id, tf_idf_score))
    doc_tf_idf_scores_cuisine[doc_id].sort(key=lambda x: x[0])

In [5]:
doc_tf_idf_scores_name

In [8]:
def top_k_restaurants_enanched(query, k=5):
    '''
    Find the top k restaurants matching the query by aggregating similarity scores across fields.
    '''
    # Process the query for each field
    query_tokens_name = list(functions.preprocessing(query))
    query_tokens_city = list(functions.preprocessing(query))
    query_tokens_cuisine = list(functions.preprocessing(query))

    # Initialize dictionaries to store cosine similarities for each field
    cosine_similarities_name = calculate_cosine_similarity_enanched(query_tokens_name, inverted_index_name, vocabulary_dict_name, doc_tf_idf_scores_name)
    cosine_similarities_city = calculate_cosine_similarity_enanched(query_tokens_city, inverted_index_city, vocabulary_dict_city, doc_tf_idf_scores_city)
    cosine_similarities_cuisine = calculate_cosine_similarity_enanched(query_tokens_cuisine, inverted_index_cuisine, vocabulary_dict_cuisine, doc_tf_idf_scores_cuisine)

    # Aggregate scores from all fields
    aggregated_scores = defaultdict(float)
    for doc_id in cosine_similarities_name:
        aggregated_scores[doc_id] += cosine_similarities_name.get(doc_id, 0)
    for doc_id in cosine_similarities_city:
        aggregated_scores[doc_id] += cosine_similarities_city.get(doc_id, 0)
    for doc_id in cosine_similarities_cuisine:
        aggregated_scores[doc_id] += cosine_similarities_cuisine.get(doc_id, 0)

    # Sort aggregated scores in descending order
    sorted_docs = sorted(aggregated_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-k results
    top_k = sorted_docs[:k]
    top_k_ids = [doc_id for doc_id, _ in top_k]
    top_k_scores = [score for _, score in top_k]

    # Prepare final DataFrame with restaurant information and similarity scores
    results_df = df.loc[top_k_ids][['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    results_df['Similarity Score'] = top_k_scores

    return results_df

def calculate_cosine_similarity_enanched(query_tokens, inverted_index, vocabulary_dict, doc_tf_idf_scores):
    '''
    Helper function to calculate cosine similarity for a single field.
    '''
    query_tf_idf = []
    n = len(doc_tf_idf_scores)  # total number of documents

    # Compute TF-IDF for query terms
    for term in query_tokens:
        term_id = vocabulary_dict.get(term)
        if term_id is None:
            continue
        n_term = len(inverted_index[term_id])
        IDF = np.log10(n / n_term) if n_term else 0
        TF = query_tokens.count(term)
        query_tf_idf.append((term_id, TF * IDF))

    # Normalize query vector
    query_norm = np.linalg.norm([score for _, score in query_tf_idf])

    # Calculate cosine similarity
    cosine_similarity = defaultdict(float)
    for doc_id in inverted_index[term_id]:
        _, doc_scores = zip(*doc_tf_idf_scores[doc_id])
        doc_norm = np.linalg.norm(doc_scores)
        intersection_scores = [score for term_id, score in query_tf_idf if term_id in doc_tf_idf_scores[doc_id]]
        cosine_similarity[doc_id] = np.dot(intersection_scores, doc_scores) / (query_norm * doc_norm) if doc_norm else 0
    
    return cosine_similarity


In [21]:
def top_k_restaurants_enanched(query, k=5, price_range=None, regions=None, credit_cards=None, facilities=None):
    '''
    Find the top k restaurants matching the query by aggregating similarity scores across fields.
    Applies additional filters for price range, region, accepted credit cards, and facilities.
    '''
    # Process the query for each field
    query_tokens_name = list(functions.preprocessing(query))
    query_tokens_city = list(functions.preprocessing(query))
    query_tokens_cuisine = list(functions.preprocessing(query))

    # Initialize dictionaries to store cosine similarities for each field
    cosine_similarities_name = calculate_cosine_similarity_enanched(query_tokens_name, inverted_index_name, vocabulary_dict_name, doc_tf_idf_scores_name)
    cosine_similarities_city = calculate_cosine_similarity_enanched(query_tokens_city, inverted_index_city, vocabulary_dict_city, doc_tf_idf_scores_city)
    cosine_similarities_cuisine = calculate_cosine_similarity_enanched(query_tokens_cuisine, inverted_index_cuisine, vocabulary_dict_cuisine, doc_tf_idf_scores_cuisine)

    # Aggregate scores from all fields
    aggregated_scores = defaultdict(float)
    for doc_id in cosine_similarities_name:
        aggregated_scores[doc_id] += cosine_similarities_name.get(doc_id, 0)
    for doc_id in cosine_similarities_city:
        aggregated_scores[doc_id] += cosine_similarities_city.get(doc_id, 0)
    for doc_id in cosine_similarities_cuisine:
        aggregated_scores[doc_id] += cosine_similarities_cuisine.get(doc_id, 0)

    # Sort aggregated scores in descending order
    sorted_docs = sorted(aggregated_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-k results
    top_k = sorted_docs[:k]
    top_k_ids = [doc_id for doc_id, _ in top_k]
    top_k_scores = [score for _, score in top_k]

    # Prepare final DataFrame with restaurant information and similarity scores
    results_df = df.loc[top_k_ids][['restaurantName', 'address', 'description', 'website', 'priceRange', 'region', 'acceptedCreditCards', 'servicesFacilities']]
    results_df['Similarity Score'] = top_k_scores

    # Apply price range filter if provided
    if price_range:
        results_df = results_df[results_df['priceRange'].isin(price_range)]

    # Apply region filter if provided
    if regions:
        results_df = results_df[results_df['region'].isin(regions)]

    # Apply accepted credit cards filter if provided
    if credit_cards:
        results_df = results_df[results_df['acceptedCreditCards'].apply(lambda x: any(card in x for card in credit_cards))]

    # Apply services and facilities filter if provided
    if facilities:
        results_df = results_df[results_df['servicesFacilities'].apply(lambda x: all(facility in x for facility in facilities))]

    return results_df


In [22]:
def calculate_cosine_similarity_enanched(query_tokens, inverted_index, vocabulary_dict, doc_tf_idf_scores):
    '''
    Helper function to calculate cosine similarity for a single field.
    '''
    query_tf_idf = []
    n = len(doc_tf_idf_scores)  # total number of documents

    # Compute TF-IDF for query terms
    for term in query_tokens:
        term_id = vocabulary_dict.get(term)
        if term_id is None:
            continue
        n_term = len(inverted_index[term_id])
        IDF = np.log10(n / n_term) if n_term else 0
        TF = query_tokens.count(term)
        query_tf_idf.append((term_id, TF * IDF))

    # Normalize query vector
    query_norm = np.linalg.norm([score for _, score in query_tf_idf])

    # Calculate cosine similarity
    cosine_similarity = defaultdict(float)
    for term_id, query_score in query_tf_idf:
        if term_id not in inverted_index:
            continue  # Skip terms not in the index
        
        # Process each document containing this term
        for doc_id, doc_score in inverted_index[term_id]:
            if doc_id not in doc_tf_idf_scores or not doc_tf_idf_scores[doc_id]:
                continue  # Skip if doc_id is missing or empty in doc_tf_idf_scores
            
            # Retrieve document scores and normalize
            _, doc_scores = zip(*doc_tf_idf_scores[doc_id])
            doc_norm = np.linalg.norm(doc_scores)
            if doc_norm == 0:
                continue  # Avoid division by zero
            
            # Calculate cosine similarity for current document
            cosine_similarity[doc_id] += (query_score * doc_score) / (query_norm * doc_norm) if query_norm else 0
            cosine_similarity[doc_id] = cosine_similarity[doc_id] / 3
            
    return cosine_similarity


In [ ]:
top_k_restaurants_enanched(
    'Filippino Lipari Seafood, Sicilian',
    k=5,
    price_range=['€', '€€€'],
    regions=['Sicilia', 'Campania'],
    credit_cards=['Visa', 'Amex'],
    facilities=['Wi-Fi', 'Terrace']
)

In [14]:
top_k_restaurants_enanched('Filippino Lipari Seafood, Sicilian')

In [23]:
df.head()

In [15]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dati simulati
cuisine_types = ["Italian", "Japanese", "Mexican", "Indian"]
facilities = ["WiFi", "Parking", "Outdoor Seating", "Pet Friendly"]
price_ranges = ["$", "$$", "$$$", "$$$$"]

# Creazione dei widget
search_input = widgets.Text(placeholder="Type to search...", description="Description:")
cuisine_dropdown = widgets.Dropdown(
    options=["All"] + cuisine_types,
    value="All",
    description="Cuisine Type:"
)
facilities_checkboxes = [widgets.Checkbox(value=False, description=facility) for facility in facilities]
price_dropdown = widgets.Dropdown(
    options=["All"] + price_ranges,
    value="All",
    description="Price Range:"
)
num_results_slider = widgets.IntSlider(value=5, min=1, max=50, step=1, description="Results:")

search_button = widgets.Button(description="Search", button_style="success")
clear_button = widgets.Button(description="Clear", button_style="danger")

# Funzioni di ricerca e di reset
def on_search_clicked(b):
    search_params = {
        "description": search_input.value,
        "cuisine_type": cuisine_dropdown.value,
        "facilities_services": [cb.description for cb in facilities_checkboxes if cb.value],
        "price_range": price_dropdown.value,
        "num_results": num_results_slider.value
    }
    clear_output()
    display_widgets()
    print("Search Parameters:", search_params)

def on_clear_clicked(b):
    search_input.value = ""
    cuisine_dropdown.value = "All"
    for cb in facilities_checkboxes:
        cb.value = False
    price_dropdown.value = "All"
    num_results_slider.value = 5
    clear_output()
    display_widgets()

# Collegamento dei pulsanti con le funzioni
search_button.on_click(on_search_clicked)
clear_button.on_click(on_clear_clicked)

# Funzione per visualizzare tutti i widget
def display_widgets():
    display(
        widgets.VBox([
            search_input,
            cuisine_dropdown,
            widgets.VBox(facilities_checkboxes),
            price_dropdown,
            num_results_slider,
            widgets.HBox([search_button, clear_button])
        ])
    )

# Mostrare l'interfaccia
display_widgets()


In [33]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import geopandas as gpd
import pandas as pd
import json

# Carica i dati dei ristoranti e delle fasce di prezzo
restaurants_df = pd.read_csv('geocode.tsv', sep="\t", usecols=['restaurantName', 'region', 'latitude', 'longitude'])
priceRangeDF = pd.read_csv('restaurants_data.tsv', sep="\t", usecols=['restaurantName', 'priceRange'])
restaurants_df = restaurants_df.merge(priceRangeDF, on='restaurantName', how='right')

# Carica il file GeoJSON delle regioni italiane
gdf = gpd.read_file("./GeoJson/limits_IT_regions.geojson")
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Funzione per eseguire la ricerca personalizzata dei ristoranti (simulazione)
def custom_search_restaurants(region_name, price_range=None):
    # Filtro per regione e, se specificato, per fascia di prezzo
    filtered_df = restaurants_df[restaurants_df['region'] == (" " + region_name)]
    if price_range:
        filtered_df = filtered_df[filtered_df['priceRange'] == price_range]
    return filtered_df

# Crea l'app Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    # Mappa Italia e selezione checkbox
    html.Div([
        dcc.Graph(id='italy-map', clickData=None, style={'width': '50vw', 'height': '100vh'}),
        dcc.Graph(id='region-map', style={'width': '50vw', 'height': '100vh'})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    # Checkbox e pulsante per la visualizzazione dei ristoranti
    html.Div([
        dcc.Checklist(
            id='show-restaurants',
            options=[{'label': 'Mostra Ristoranti', 'value': 'show'}],
            value=[],
            labelStyle={'display': 'inline-block'}
        ),
        html.Button('Cerca Ristoranti', id='search-restaurants-btn', n_clicks=0),
    ], style={'textAlign': 'center', 'padding': '10px'})
])

# Callback per la mappa dell'Italia
@app.callback(
    Output('italy-map', 'figure'),
    Input('italy-map', 'clickData')
)
def update_italy_map(clickData):
    selected_region = None
    if clickData:
        selected_region = clickData['points'][0]['location']
    
    fig = px.choropleth_mapbox(
        geojson=json.loads(gdf.to_json()),
        locations=gdf['reg_name'],
        featureidkey="properties.reg_name",
        color_discrete_sequence=['grey'],
        opacity=0.2,
        mapbox_style="carto-positron",
        zoom=4.5,
        center={"lat": 41.8719, "lon": 12.5674},
        title="Mappa delle Regioni Italiane"
    )
    
    if selected_region:
        fig.update_traces(marker_line_color="red", selector=dict(location=selected_region))
    
    return fig

# Callback per la mappa della regione selezionata e la visualizzazione dei ristoranti
@app.callback(
    Output('region-map', 'figure'),
    [Input('italy-map', 'clickData'),
     Input('show-restaurants', 'value'),
     Input('search-restaurants-btn', 'n_clicks')],
    [State('italy-map', 'clickData')]
)
def display_selected_region(clickData, show_restaurants, n_clicks, clickData_state):
    # Determina se mostrare tutti i ristoranti o filtrare i risultati di ricerca
    if not clickData:
        return {}

    selected_region = clickData['points'][0]['location']
    region_gdf = gdf[gdf['reg_name'] == selected_region]
    region_gdf_proj = region_gdf.to_crs(epsg=32632)
    centroid_proj = region_gdf_proj.geometry.centroid.iloc[0]
    centroid = gpd.GeoSeries([centroid_proj], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
    center_lat, center_lon = centroid.y, centroid.x

    # Usa custom_search_restaurants solo se il pulsante "Cerca Ristoranti" è stato cliccato
    if n_clicks > 0:
        filtered_restaurants = custom_search_restaurants(selected_region)
    else:
        filtered_restaurants = restaurants_df[restaurants_df['region'] == (" " + selected_region)]

    fig = px.choropleth_mapbox(
        geojson=json.loads(region_gdf.to_json()),
        locations=[selected_region],
        featureidkey="properties.reg_name",
        mapbox_style="carto-positron",
        color_discrete_sequence=['red'],
        opacity=0.1,
        zoom=6,
        center={"lat": center_lat, "lon": center_lon},
        title=f"Mappa di {selected_region} con Ristoranti"
    )

    if 'show' in show_restaurants:
        for price, color in zip(["€", "€€", "€€€", "€€€€"], ["blue", "green", "purple", "brown"]):
            price_restaurants = filtered_restaurants[filtered_restaurants['priceRange'] == price]
            fig.add_scattermapbox(
                lat=price_restaurants['latitude'].tolist(),
                lon=price_restaurants['longitude'].tolist(),
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.8),
                text=price_restaurants['restaurantName'],
                name=price,
                hovertemplate=(
                    "<b>Nome Ristorante:</b> %{text}<br>"
                    "<b>Latitudine:</b> %{lat}<br>"
                    "<b>Longitudine:</b> %{lon}<br>"
                )
            )

    fig.update_layout(
        mapbox=dict(
            layers=[
                {
                    'source': json.loads(region_gdf.to_json()), 
                    'type': "line",
                    'color': "black",
                    'line': {'width': 0.5}
                }
            ]
        )
    )
    
    return fig

# Avvia l'app Dash
if __name__ == '__main__':
    app.run_server(jupyter_mode = 'external', debug=True)


In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from itertools import chain

class RestaurantSearchInterface:
    def __init__(self, df, vocabulary_df, inverted_index):
        """
        Initialize the search interface with the enhanced search engine.
        """
        self.df = df
        self.vocabulary_df = vocabulary_df
        self.inverted_index = inverted_index
        
        self.cuisine_types = ['All Cuisines'] + sorted(df['cuisineType'].dropna().unique().tolist())
        self.facilities = sorted(list(set(chain(*df['facilitiesServices'].apply(eval)))))
        self.price_ranges = ['All Prices'] + sorted(df['priceRange'].dropna().unique().tolist())
        
        # Create widgets
        self.create_widgets()
        
    def create_widgets(self):
        # Search text input
        self.search_text = widgets.Text(
            description='Search:',
            placeholder='Enter keywords (e.g., Greek, pasta...)',
            layout=widgets.Layout(width='70%')
        )
        
        # Number of results slider
        self.results_select = widgets.IntSlider(
            min=1,
            max=20,
            step=1,
            value=5,
            description='Show top:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%')
        )

        # Cuisine type dropdown
        self.cuisine_select = widgets.Dropdown(
            options=self.cuisine_types,
            description='Cuisine:',
            value='All Cuisines',
            layout=widgets.Layout(width='70%')
        )
        
        # Price range dropdown
        self.price_select = widgets.Dropdown(
            options=self.price_ranges,
            description='Price:',
            value='All Prices',
            layout=widgets.Layout(width='70%')
        )
        
        # Facilities checkboxes
        self.facilities_boxes = [
            widgets.Checkbox(
                value=False,
                description=facility,
                layout=widgets.Layout(width='auto')
            ) for facility in self.facilities
        ]
        
        # Facilities container
        self.facilities_box = widgets.VBox([
            widgets.HTML("<h4 style='margin:10px 0;'>Select Facilities:</h4>"),
            widgets.Box(self.facilities_boxes, layout=widgets.Layout(display='flex', flex_wrap='wrap'))
        ])
        
        # Search and clear buttons
        self.search_button = widgets.Button(
            description='🔍 Search',
            button_style='success',
            layout=widgets.Layout(width='30%', margin='10px 5px')
        )
        self.clear_button = widgets.Button(
            description='✖ Clear',
            button_style='danger',
            layout=widgets.Layout(width='30%', margin='10px 5px')
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Button actions
        self.search_button.on_click(self.handle_search)
        self.clear_button.on_click(self.handle_clear)

    def format_results_as_table(self, results):
        """Format search results as an HTML table with CSS styling."""
        table_style = """
        <style>
        .search-results { width: 100%; border-collapse: collapse; margin: 20px 0; font-size: 15px; }
        .search-results th, .search-results td { padding: 12px; text-align: left; border: 1px solid #ddd; }
        .search-results th { background-color: #4CAF50; color: white; }
        .search-results tr:nth-child(even) { background-color: #f2f2f2; }
        .search-results tr:hover { background-color: #ddd; }
        </style>
        """
        
        html = table_style + '<table class="search-results">'
        headers = ['Name', 'Address', 'Description', 'Website', 'Score']
        html += '<tr>' + ''.join([f'<th>{h}</th>' for h in headers]) + '</tr>'
        for _, row in results.iterrows():
            html += '<tr>'
            html += f'<td>{row["restaurantName"]}</td>'
            html += f'<td>{row["address"]}</td>'
            desc = row['description'][:150] + '...' if len(row['description']) > 150 else row['description']
            html += f'<td>{desc}</td>'
            html += f'<td>{row["website"]}</td>'
            html += f'<td>{row["customScore"]:.3f}</td>'
            html += '</tr>'
            
        html += '</table>'
        return html

    def handle_search(self, button):
        """Handle the search button click."""
        with self.output:
            clear_output()
            
            #query = self.search_text.value.strip()
            #if not query:
            #    display(HTML("<p style='color:red;'>Please enter a search query.</p>"))
            #    return
                
            #cuisine = None if self.cuisine_select.value == 'All Cuisines' else self.cuisine_select.value
            #price_range = None if self.price_select.value == 'All Prices' else self.price_select.value
            
            #facilities = [f.description for f in self.facilities_boxes if f.value]
            #facilities = facilities if facilities else None
            
            #k = self.results_select.value
            
            try:
                # Raccolta dei valori inseriti dall'utente
                query = {
                    'description': self.search_text.value.strip(),
                    'cuisineType': None if self.cuisine_select.value == 'All Cuisines' else self.cuisine_select.value,
                    'facilitiesServices': [f.description for f in self.facilities_boxes if f.value],
                    'priceRange': None if self.price_select.value == 'All Prices' else self.price_select.value,
                    'num_results': self.results_select.value
                }

                # Chiamata alla funzione di ricerca
                results = functions.find_top_custom_restaurants(
                    query,
                    self.vocabulary_df,
                    self.inverted_index,
                    self.df
                )
                if results.empty:
                    display(HTML("<p style='color:orange;'>No results found.</p>"))
                else:
                    display(HTML(f"<p style='color:green;'>Found {len(results)} results:</p>"))
                    display(HTML(self.format_results_as_table(results)))
                    
            except Exception as e:
                display(HTML(f"<p style='color:red;'>An error occurred: {str(e)}</p>"))
    
    def handle_clear(self, button):
        """Clear all fields and reset the output area."""
        self.search_text.value = ''
        self.cuisine_select.value = 'All Cuisines'
        self.price_select.value = 'All Prices'
        for checkbox in self.facilities_boxes:
            checkbox.value = False
        self.results_select.value = 5
        with self.output:
            clear_output()
    
    def display_interface(self):
        """Display the complete search interface."""
        search_box = widgets.VBox([
            widgets.HTML("<h2 style='text-align:center; color:#4CAF50;'>Restaurant Search</h2>"),
            widgets.HBox([self.search_text, self.results_select]),
            self.cuisine_select,
            self.price_select,
            self.facilities_box,
            widgets.HBox([self.search_button, self.clear_button]),
            self.output
        ])
        
        display(search_box)


In [10]:
search_ui = RestaurantSearchInterface(df, vocabulary_df, inverted_index)
search_ui.display_interface()

# Algorithmic Question (AQ)

## PseudoCode

The intuition is that, starting from the origin (0, 0) in the first quadrant where all packages are located, the first package is always reachable. Given that we can only move up or right, we can only reach packages that are above or to the right of the current one. Therefore, for each package reached, we need to check the next one: if it is below or to the left, we can print "NO" because it’s unreachable. Otherwise, we proceed, but to ensure the lexicographically smallest path, we need to sort all packages in ascending order by their coordinates.

for each test case (t): read n

packages = []

for i = 1 to n: O(n) read x[i], y[i] append (x[i], y[i]) to packages

sort packages by x, then by y O(n Log n)

x_current, y_current = 0, 0 path = '' possible = True

for x[i], y[i] in packages: O(n) if x[i] < x_current or y[i] < y_current: possible = False break num_right_moves = x[i] - x_current num_up_moves = y[i] - y_current

path += "R" * num_right_moves path += "U" * num_up_moves

x_current = x[i] y_current = y[i]

if possible: print("YES") print(path) else: print("NO")